In [7]:
!pip install web3 requests pandas tqdm

import pandas as pd
import requests
from tqdm import tqdm
from web3 import Web3


In [8]:
wallets_df = pd.read_csv("Wallet id.csv")
wallets = wallets_df['wallet_id'].tolist()
wallets[:5]  


['0x0039f22efb07a647557c7c5d17854cfd6d489ef3',
 '0x06b51c6882b27cb05e712185531c1f74996dd988',
 '0x0795732aacc448030ef374374eaae57d2965c16c',
 '0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9',
 '0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae']

In [9]:
COMPOUND_SUBGRAPH_URL = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"


In [10]:
def query_subgraph(query):
    response = requests.post(COMPOUND_SUBGRAPH_URL, json={'query': query})
    try:
        json_data = response.json()
    except ValueError:
        raise Exception(f"Non-JSON response: {response.text}")

    if 'errors' in json_data:
        raise Exception(f"GraphQL Error: {json_data['errors']}")
    if 'data' not in json_data:
        raise Exception(f"'data' field missing in response: {json_data}")
    return json_data


In [11]:
def get_wallet_data(wallet):
    query = f"""
    {{
      account(id: "{wallet.lower()}") {{
        id
        tokens {{
          symbol
          supplyBalanceUnderlying
          borrowBalanceUnderlying
        }}
        totalBorrowValueInEth
        totalCollateralValueInEth
        health
      }}
    }}
    """
    try:
        result = query_subgraph(query)
        acc = result["data"]["account"]
        if not acc:
            return {
                "wallet_id": wallet,
                "borrow_eth": 0,
                "collateral_eth": 0,
                "health": 0,
                "supplied_assets": 0,
                "borrowed_assets": 0,
            }

        tokens = acc["tokens"]
        return {
            "wallet_id": wallet,
            "borrow_eth": float(acc.get("totalBorrowValueInEth", 0)),
            "collateral_eth": float(acc.get("totalCollateralValueInEth", 0)),
            "health": float(acc.get("health", 0)),
            "supplied_assets": sum(float(t["supplyBalanceUnderlying"]) for t in tokens),
            "borrowed_assets": sum(float(t["borrowBalanceUnderlying"]) for t in tokens),
        }

    except Exception as e:
        print(f"Error for wallet {wallet}: {e}")
        return {
            "wallet_id": wallet,
            "borrow_eth": 0,
            "collateral_eth": 0,
            "health": 0,
            "supplied_assets": 0,
            "borrowed_assets": 0,
        }


In [12]:
wallet_data = []
for wallet in tqdm(wallets):
    data = get_wallet_data(wallet)
    wallet_data.append(data)

wallet_df = pd.DataFrame(wallet_data)
wallet_df.head()


  1%|▊                                                                                 | 1/103 [00:02<03:55,  2.31s/it]

Error for wallet 0x0039f22efb07a647557c7c5d17854cfd6d489ef3: GraphQL Error: [{'message': 'This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com'}]


  2%|█▌                                                                                | 2/103 [00:03<03:05,  1.84s/it]

Error for wallet 0x06b51c6882b27cb05e712185531c1f74996dd988: GraphQL Error: [{'message': 'This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com'}]


  3%|██▍                                                                               | 3/103 [00:05<02:44,  1.65s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▏| 102/103 [02:35<00:01,  1.46s/it]

Error for wallet 0xf80a8b9cfff0febf49914c269fb8aead4a22f847: GraphQL Error: [{'message': 'This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com'}]


100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [02:37<00:00,  1.53s/it]

Error for wallet 0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac: GraphQL Error: [{'message': 'This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com'}]


,wallet_id,borrow_eth,collateral_eth,health,supplied_assets,borrowed_assets
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0,0,0,0,0
1,0x06b51c6882b27cb05e712185531c1f74996dd988,0,0,0,0,0
2,0x0795732aacc448030ef374374eaae57d2965c16c,0,0,0,0,0
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,0,0,0,0,0
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,0,0,0,0,0


In [14]:
def generate_fallback_data(wallet):
    # Simulate wallet-level Compound stats
    health = round(np.random.uniform(0.5, 3.0), 2)
    collateral_eth = round(np.random.uniform(0, 50), 2)
    borrow_eth = round(collateral_eth * np.random.uniform(0, 1.2), 2)
    supplied_assets = round(collateral_eth * np.random.uniform(0.8, 1.2), 2)
    borrowed_assets = round(borrow_eth * np.random.uniform(0.9, 1.1), 2)

    return {
        "wallet_id": wallet,
        "borrow_eth": borrow_eth,
        "collateral_eth": collateral_eth,
        "health": health,
        "supplied_assets": supplied_assets,
        "borrowed_assets": borrowed_assets
    }

# Generate fallback for all wallets
wallet_data = []
for wallet in tqdm(wallets):
    wallet_data.append(generate_fallback_data(wallet))

wallet_df = pd.DataFrame(wallet_data)
wallet_df.head()


100%|████████████████████████████████████████████████████████████████████████████████████████| 103/103 [00:00<?, ?it/s]


,wallet_id,borrow_eth,collateral_eth,health,supplied_assets,borrowed_assets
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,23.37,38.65,2.70,36.99,24.48
1,0x06b51c6882b27cb05e712185531c1f74996dd988,30.65,30.80,1.29,33.12,28.84
2,0x0795732aacc448030ef374374eaae57d2965c16c,22.04,43.54,0.91,50.05,21.52
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,1.69,38.99,0.62,33.04,1.62
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,46.74,41.33,0.81,48.02,48.43


In [45]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 435.7 kB/s eta 0:00:04
   ---- ----------------------------------- 0.2/1.5 MB 1.3 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.5 MB 2.0 MB/s eta 0:00:01
   -------------- ------------------------- 0.5/1.5 MB 2.5 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 2.7 MB/s eta 0:00:01
   ------------------------ --------------- 0.9/1.5 MB 2.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.1/1.5 MB 2.9 MB/s eta 0:00:01
   --------------------------------- ------ 1.2/1.5 MB 3.0 MB/s eta 0:00:01
   -------------------------------------- - 1.4/1.5 MB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.0 MB/s eta 0:00:00


In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score


In [55]:
def simulate_risk_score(row):
    # Higher borrow/collateral ratio, lower health => higher risk
    score = (
        (1 - min(row["health"] / 3.0, 1)) * 400 +  
        min(row["borrow_eth"] / (row["collateral_eth"] + 1e-6), 1) * 300 +
        min(row["borrowed_assets"] / (row["supplied_assets"] + 1e-6), 1) * 300
    )
    return round(min(score, 1000), 2)

wallet_df["risk_score"] = wallet_df.apply(simulate_risk_score, axis=1)
wallet_df.head()


,wallet_id,borrow_eth,collateral_eth,health,supplied_assets,borrowed_assets,risk_score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,23.37,38.65,2.70,36.99,24.48,419.94
1,0x06b51c6882b27cb05e712185531c1f74996dd988,30.65,30.80,1.29,33.12,28.84,787.77
2,0x0795732aacc448030ef374374eaae57d2965c16c,22.04,43.54,0.91,50.05,21.52,559.52
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,1.69,38.99,0.62,33.04,1.62,345.05
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,46.74,41.33,0.81,48.02,48.43,892.00


In [56]:
features = ["borrow_eth", "collateral_eth", "health", "supplied_assets", "borrowed_assets"]
X = wallet_df[features]
y = wallet_df["risk_score"]

# Normalize features (optional but helps)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [57]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [58]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [60]:
y_pred = model.predict(X_test)

print("R² Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))




R² Score: 0.7390127847033894
RMSE: 101.62796895552546


In [61]:
wallet_df["predicted_risk_score"] = model.predict(scaler.transform(X))
wallet_df["predicted_risk_score"] = wallet_df["predicted_risk_score"].clip(0, 1000).round(2)

wallet_df[["wallet_id", "predicted_risk_score"]].to_csv("wallet_risk_scores_ml.csv", index=False)


In [62]:
wallet_df["borrow_to_collateral"] = wallet_df["borrow_eth"] / (wallet_df["collateral_eth"] + 1e-6)
wallet_df["borrow_to_supply"] = wallet_df["borrowed_assets"] / (wallet_df["supplied_assets"] + 1e-6)
wallet_df["net_exposure"] = wallet_df["borrow_eth"] - wallet_df["collateral_eth"]
wallet_df["utilization"] = wallet_df["borrow_eth"] / (wallet_df["borrow_eth"] + wallet_df["collateral_eth"] + 1e-6)


In [84]:
# Use only the original 5 features used during training
features = [
    "borrow_eth", 
    "collateral_eth", 
    "health", 
    "supplied_assets", 
    "borrowed_assets"
]

In [64]:
model = RandomForestRegressor(
    n_estimators=300,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42
)


In [66]:
!pip install xgboost


   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB 653.6 kB/s eta 0:03:50
   ---------------------------------------- 0.0/150.0 MB 653.6 kB/s eta 0:03:50
   ---------------------------------------- 0.0/150.0 MB 653.6 kB/s eta 0:03:50
   ---------------------------------------- 0.0/150.0 MB 653.6 kB/s eta 0:03:50
   ---------------------------------------- 0.1/150.0 MB 409.6 kB/s eta 0:06:06
   ---------------------------------------- 0.1/150.0 MB 425.1 kB/s eta 0:05:53
   ---------------------------------------- 0.1/150.0 MB 425.1 kB/s eta 0:05:53
   -----------

In [67]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

print("R² Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))


R² Score: 0.8432804032886826
RMSE: 78.7526691520918


In [87]:
wallet_df[["wallet_id", "predicted_risk_score"]].to_csv("xgb_wallet_risk_scores.csv", index=False)
